# Factorization machines

Привет, друзья! Я хотел бы рассказать вам немного о концепте факторизационных машин, который в свое время меня впечатлил. Здесь не будет много строгой математики, однако мы рассмотрим предпосылки к появлению этой модели, и ее математические основы, а во второй части, конечно, немного попрактикуемся.

На написание этого небольшого с позволения сказать тьюториала меня вдохновили и помогли [вот эта статья](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf) и [лекции Евгения Соколова](https://www.youtube.com/watch?v=YmDT1hUTv2E) в CSClub.

## Немного теории и предпосылок
Хотя факторизационные машины могут применяться во всех задачах наравне с известными читателю моделями (линейными, деревьями, и т.д.), их основное достоинство - работа в условиях высокой разреженности данных (в дальнейшем станет понятно, почему). Поэтому, рассмотрим эту модель на примере задачи построения рекомендательной системы, реализации которых разной степени успешности вы можете встретить практически на любом ресурсе в наше время.
<img src="../../img/fmtutor_rseverywhere.jpeg">

 Немного вне темы данной статьи скажу, что она будет относиться к так называемым [Контентным](https://en.wikipedia.org/wiki/Recommender_system#Content-based_filtering) моделям, которые в отличие от [Колаборативных](https://en.wikipedia.org/wiki/Recommender_system#Collaborative_filtering) моделей, допускают и опираются на дополнительные знания о товаре или покупателе, например, описание или историю посещений/покупок.

Итак, рассмотрим задачу рекомендации пользователям фильмов на некоем абстрактном ресурсе. Все просто - пользователи смотрят фильмы, и ставят им оценки от 1 до 5. Мы же хотим предсказывать оценку для новой пары пользователь-фильм, и, соответственно, если она высока, мы этот фильм пользователю рекомендуем. 

Закодируем информацию (создадим объекты выборки) следующим образом. Каждый пример будет вектором, условно поделенным на несколько частей:
1. Идентификатор пользователя, являющийся результатом применения OheHot к идентификационному номеру пользователя, то есть это вектор, одна из координат которого равна 1, остальные - 0
2. Идентификатор фильма - аналогично идентификатору пользователя из п. 1
3. Любая другая полезная информация, как например: история просмотров, оценки другим фильмам, и проч.

Рассмотрим конкретнее:

$ U = \{Alice (A), Bob (B), Charlie (C), . . .\} $ - множество пользователей

$ I = \{Titanic (TI) ,Notting Hill (NH),Star Wars (SW),Star Trek (ST), . . .\} $ - множество фильмов

$ S = \{(A,TI,2010-1,5),(A,NH,2010-2,3),(A,SW,2010-4,1),(B,SW,2009-5,4),
(B,ST,2009-8,5), . . . \} $ - имеющиеся данные

В виде матрицы:
<img src="../../img/fmtutor_table.png">
Здесь синяя часть - активный юзер в данной транзакции, оранжевая - активный фильм, желтая - остальные фильмы, просмотренные юзером, зеленая - время в месяцах, начиная с января 2009 года, бордовая - последний просмотренный фильм. Не обязательно кодировать дополнительные признаки именно так, но важно следить, чтобы порядки значений не очень сильно отличались, потому что факторизационная машина многим похожа на линейную регрессию, а как читатель помнит из одной из статей курса, для такой модели важно, чтобы компоненты были примерно одного порядка.

### Собственно, модель

Итак, мы хотим обучить модель на этих данных. Фильмов у нас в базе много, а пользователей еще больше, мы ведь разрабатываем систему для популярного портала, не так ли? Поэтому на самом деле данные очень разрежены, и читатель уже знает, что, например, деревья плохо справятся с такой задачей, а вот линейные модели не должны встретить особых трудностей.

Итак, первая наша идея - простая линейная модель:
$$ \hat{y} =  w_{0} + \sum_{i = 1}^{n}w_{i}x_{i} $$

Проблема такой модели в том, что она <b>не улавливает взаимодействий</b> между нашими сущностями. То есть простая линейная регрессия запомнит только то, что, скажем, Боб ставит обычно высокую оценку, а фильм NittingHill получает средние оценки.

Мы же хотим, чтобы модель выучивала еще и связи между людьми, фильмами и так далее. Ну так давайте их сделаем!
$$ \hat{y} = w_{0} + \sum_{1}^{n}w_{i}x_{i} + \sum_{i=1}^n\sum_{j=i+1}^{n}w_{i,j}x_{i}x_{j} $$

Уже лучше, модель должна выучить связи между всеми компонентами вектора примера. Однако внимательный читатель
заметит основной недостаток такой модели - <b>число весов $ w_{i,j} $ растет пропорционально квадрату числа компонент в векторе</b>. Это большая проблема, потому что, например, число юзеров может запросто быть порядка миллиона, и, соответственно, порядок числа весов будет $ 10^{12} $, ну или еще больше. Это непозволительный расход памяти (если ее вообще столько найдется!)

Тогда упростим ее немного так, чтобы она не потеряла способности выучивать взаимодействия между компонентами, но в то же время не обладала таким гигантским числом параметров:
$$ \hat{y} = w_{0} + \sum_{1}^{n}w_{i}x_{i} + \sum_{i=1}^n\sum_{j=i+1}^{n}\langle v_{i}, v_{j} \rangle x_{i}x_{j},$$
где $$ w_{0} \in \mathbb{R}, w_{i} \in \mathbb{R}^{n}, v_{i} \in \mathbb{R}^{n\times k},$$ a $ \langle v_{i}, v_{j} \rangle  = \sum_{f=1}^{k}v_{i,f}\cdot v_{j,f}$ - покомпонентное произведение векторов. (Да, для двух векторов это - скалярное произведение, но в дальнейшем мы обобщим это на случай большего числа векторов, поэтому используем это обозначение, и не будем его так называть)

Итак, выше написано выражение для модели факторизационной машины степени $d = 2$, гиперпараметр $k$ отвечает за размерность факторизации. Степень 2 говорит о том, что модель выучивает одиночные и парные взаимодействия между компонентами вектора примеров.

На самом деле, теоретический факт из линейной алгебры говорит о том, что при достаточно большом числе $k$ можно выразить все взаимодействия в исходной матрице, однако на практике лучше использовать небольшие значения этого гиперпараметра, так как во-первых, зачастую данных недостаточно для выражения сложных взаимодействий, а во-вторых ограничение $k$ уменьшает выразительную силу модели, и, как следствие, препятствует ее переобучению.

Следующий замечательный факт состоит в том, что вычисляется эта функция за $O(kn)$. Доказательство этого факта не сложно, но не хочется перегружать статью формулами, желающие могут найти его в [статье](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf).

Заметим, что модель легко обобщается на случай произвольного числа степеней:
$$ \hat{y} = w_{0} + \sum_{1}^{n}w_{i}x_{i} + \sum_{l=2}^{d}\sum_{i_{1} = 1}^{n}...\sum_{i_{} = i_{l-1}+1}^{n}\left ( \prod_{j=1}^{l} x_{i_{j}}\right )\left ( \sum_{f=1}^{k_{l}}\prod_{j=1}^{l}v_{i_{j}, f}^{(l)} \right ) $$
На практике же обычно взаимодействия выше третьего порядка редко когда могут понадобиться, мне с ходу и не придумать такой пример :)

Если вернуться к нашей задаче с рекомендательно системой, то нам остается только выбрать гиперпараметры для модели (порядок и размерность), и предсказывать оценку как для обычной регрессионной задачи.

Факторизационные машины можно использовать не только в задаче регрессии, но и для других, среди которых:
1. Задача бинарной классификации (которая обобщается на случай многих классов элементарно с помощью алгоритмов OneVSOne или OneVSAll). Принадлежность к тому или иному классу определяется знаком $\hat{y}$
2. Задача ранжирования - входные векторы сортируются по соответствующему значению $\hat{y}$

Напоследок, переходя к практике, следует заметить также, что обычно в модель добавляют регуляризацию, например, $L2$ , чтобы также избежать переобучения.

В соревнованиях на Kaggle, если задача была так или иначе связана с рекомендациями, эту модель очень часто можно встретить в решениях победителей ;)

## Практика

<img src="../../img/fmtutor_theoryvspractice.png" height="200" width="300">

Я предлагаю рассмотреть небольшой пример работы факторизационной машины. Реализаций ее довольно много, я же хочу показать ту, которая мне показалась наиболее удобной и приятной (<b>важное замечание:</b> автор не видел <b>всех</b> реализаций, и еще не работал с реализаций FM от Vowpal Wabbit, и не исключает, что она тоже хороша!)

Единственный нюанс состоит в том, что реализация, которую я хочу показать требует установленной библиотеки [TensorFlow](https://github.com/tensorflow/tensorflow). Допускаю, что некоторые читатели еще не работали с ней - в рамках курса это нигде не потребовалось, но хорошая новость заключается в том, что сейчас нам не нужно разбираться, как она работает, ее просто нужно поставить. Автор хочет заверить читателя, что работать с этой библиотекой приется практически наверняка, если читатель планирует продолжать свои изыскания в области машинного обучения, поэтому ее установка не будет пустой тратой времени и ресурсов. Сейчас же она просто понадобится как бэкенд, поверх которого будеть работать наша факторизационная машина
<img src="../../img/fmtutor_keras_backend.jpg" height="100" width="200">

Сама же реализация факторизационной машины [вот](https://github.com/geffy/tffm)

Все вышеперечисленные библиотеки можно установать с помощью пакетного менеджера pip.

Там же, в репозитории можно найти и пример, который мы сейчас рассмотрим. Мне не хотелось бы перегружать этот тьюториал большими данными, поэтому мы оставим такие задачи, как построение рекомендательной системы пользователю для самостоятельной практики, автор утверждает, что это возможно и весьма интересно:) А рассмотрим мы известный читаетелю датасет MNIST, и посмотрим, как модель будет различать, например тройки и пятерки

#### Внимание!
Если запуск следующей клетки приводит к ошибке AttributeError: module 'pandas' has no attribute 'computation', выполните команду <br>`pip install dask --upgrade` (или `conda update dask`). Или просто запустите ее еще раз - по какой-то причине так тоже работает

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tffm import TFFMClassifier

/home/sergey/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/")

mnist_images = mnist.train.images
mnist_labels = mnist.train.labels

n_three, n_five = sum(mnist_labels==3), sum(mnist_labels==5)

X_all = np.vstack([
    mnist_images[mnist_labels==3,:],
    mnist_images[mnist_labels==5,:]
])

y_all = np.array([1]*n_three + [0]*n_five)
# make it more sparse
X_all = X_all * (np.random.uniform(0, 1, X_all.shape) > 0.8)

print('Dataset shape: {}'.format(X_all.shape))
print('Non-zeros rate: {:.05f}'.format(np.mean(X_all != 0)))
print('Classes balance: {:.03f} / {:.03f}'.format(np.mean(y_all==0), np.mean(y_all==1)))

X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, random_state=42, test_size=0.3)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Dataset shape: (10625, 784)
Non-zeros rate: 0.04040
Classes balance: 0.469 / 0.531


Мы подготовили данные, давайте теперь посмотрим, как уже известные нам модели справляются с такой задачей, они будут нашими бейзлайнами:

In [3]:
for model in [
                LogisticRegression(), 
                RandomForestClassifier(n_jobs=-1, n_estimators=200)
            ]:
    model.fit(X_tr, y_tr)
    predictions = model.predict(X_te)
    acc = accuracy_score(y_te, predictions)
    print('model: {}'.format(model.__str__()))
    print('accuracy: {}'.format(acc))
    print()

model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
accuracy: 0.8795483061480552

model: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
accuracy: 0.8848808030112923



Посмотрим теперь, как справится фактоиризационная машина с этой задачей. Эта реализация поддерживает dense и sparse формат входа, мы рассмотрим второй, так как первый в принципе на него заменить можно, а вот наоборот в задачах с большими разреженными данными будет очень сложно

In [4]:
import scipy.sparse as sp
# only CSR format supported
X_tr_sparse = sp.csr_matrix(X_tr)
X_te_sparse = sp.csr_matrix(X_te)

Создадим модель FM третьего порядка (то есть улавливающую взаимодействия между единичными, парами и тройками объектов, в нашем случае - битов) и рангом (размерностью факторизации) - 30. Читателю предлагается поэкспериментировать с этими параметрами и посмотреть на изменение метрики и скорости обучения. Также в качестве гиперпараметров мы передаем число эпох, коэффициент регуляризации и т.д. 

In [5]:
order = 3
model = TFFMClassifier(
    order=order, 
    rank=30, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=50, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    seed=42
)
model.fit(X_tr_sparse, y_tr, show_progress=True)
predictions = model.predict(X_te_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
model.destroy()

100%|██████████| 50/50 [00:06<00:00,  7.92epoch/s]

[order=3] accuracy: 0.9149937264742786


Видим, что модель неплохо справилась с задачей, по крайней мере, бейзлайн мы побили:)

Еще раз обращу внимание, что с повышением количества данных, их сложности и степени их разреженности, обычные модели станут работать намного хуже, а правильно настроенная и обученная факторизационная машина должна показывать неплохие результаты

Это все, что я хотел вам рассказать, хотя, очень многие аспекты этой темы еще можно изучать как читателю, так и автору. Если вы имеете замечания по данному материалу, пожалуйста, напишите мне в слак или на почту - sergeygorbatyuk171@gmail.com, потому что я так же как и многие из вас еще в начале пути освоения этой интересной науки, а значит, могу допускать ошибки и неточности, давайте разбираться вместе!:) 
    
    Спасибо большое, что уделили этому материалу ваше время!